In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

In [3]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [4]:
# EXTRACT HTE DATA FROM PDF
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents=loader.load()
    return documents

In [7]:
extracted_data = load_pdf("/Users/piyushpandey955/Desktop/AIML/gen ai/medical chatbot project/Medical-Chatbot/data/")

In [8]:
#extracted_data

In [9]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 3896


In [11]:
# text_chunks

In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [15]:
query_result=embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
#query_result

In [17]:

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'bhagwat-gita'
index = pc.Index(index_name)

#creating embeddings for each of the text chunks and storing
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [18]:
query="Who is Arjuna?"
docs=docsearch.similarity_search(query,k=3)
print(docs)

[Document(page_content='senses:\tvoice,\tlegs,\thands,\tthe\tanus\tand\tthe\tgenitals.\tThen,\tabove\tthe\tsenses,\nthere\tis\tthe\tmind,\twhich\tis\twithin\tand\twhich\tcan\tbe\tcalled\tthe\tsense\twithin.\nTherefore,\tincluding\tthe\tmind,\tthere\tare\televen\tsenses\taltogether.\tThen\tthere\nare\t the\t five\t objects\t of\t the\t senses:\t smell,\t taste,\t warmth,\t touch\t and\t sound.\nNow\t the\t aggregate\t of\t these\t twenty-four\t elements\t is\t called\t the\t field\t of\nactivity.\tIf\tone\tmakes\tan\tanalytical\tstudy\tof\tthese\ttwenty-four\tsubjects,\tthen'), Document(page_content="material\t bodily\t concept.\t In\t the\t bodily\t conception\t of\t life\t thereare\t specific\nduties\t for\t the\t brāhmaṇas\t and\t kṣatriyas\t respectively,\t and\t such\t duties\t are\nunavoidable.\t Svadharma\tis\tordained\tby\tthe\tLord,\tand\tthis\twill\tbe\tclarified\nin\tthe\tFourth\tChapter.\tOn\tthe\tbodily\tplane\tsvadharma\tis\tcalled\tvarṇāśrama-\ndharma,\t or\t man's\t step

In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer ,just say that you don't know ,don't try to makeup an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else

Helpful answer:
"""

In [20]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

In [21]:
llm=CTransformers(model="/Users/piyushpandey955/Desktop/AIML/gen ai/medical chatbot project/Medical-Chatbot/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8}
                )

In [22]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
import pinecone

# Use the Pinecone vectorstore from Langchain
vectorstore = Pinecone(
    index=pinecone.Index(index_name, host='https://bhagwat-gita-9d82pte.svc.aped-4627-b74a.pinecone.io'),
    embedding_function=embeddings.embed_query,
    text_key="text"
    )

# Set up the retriever with Pinecone and Langchain
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [35]:
user_input = "who is Duryodhana?"
query_embedding = embeddings.embed_query(user_input)
len(query_embedding)


384

In [36]:

# Query Pinecone using the keyword arguments format
result = index.query(vector=query_embedding, top_k=2, include_metadata=True)

# Check if results are found
if result and 'matches' in result:
    # Loop through the top results
    for match in result['matches']:
        # Extract metadata if available (assuming text is stored in metadata)
        if 'metadata' in match and 'text' in match['metadata']:
            # Extract and print the relevant text
            print("Match:", match['metadata']['text'])
        else:
            print("No metadata or text field found in this match.")
else:
    print("No results found.")

Match: rasanaṁ	ghrāṇam	eva	ca
adhiṣṭhāya	manaś	cāyaṁ
viṣayān	upasevate
śrotram—ears;	 cakṣuḥ—eyes;	 sparśanam—touch;	 ca—also;	 rasanam—
tongue;	 ghrāṇam—smelling	power;	 eva—also;	 ca—and;	 adhiṣṭhāya—being
situated;	 manaḥ—mind;	 ca—also;	 ayam—this;	 viṣayān—sense	 objects;
upasevate—enjoys.
TRANSLATION
The	living	entity,	thus	taking	another	gross	body,	obtains	a	certain
type	 of	 ear,	 tongue,	 and	 nose	 and	 sense	 of	 touch,	 which	 are	 grouped
Match: anus	 and	 the	 genital.	 The	 living	 entity	 in	 his	 conditioned	 stage	 identifies
himself	with	the	body,	but	when	he	identifies	himself	with	the	Lord	within
